## A recursive formula for $B(n)$

We can use the following [result from probability theory](https://en.wikipedia.org/wiki/Law_of_total_expectation#Proof_of_partition_formula) -- if $X$ is a random variable on a sample space $\Omega$ and ${A_i}$ is a finite partition of $\Omega$ then

$$\begin{equation} E[X] = \sum_{i}E[X| A_i]P(A_i) \end{equation}$$

First, let $X$ be the number of guesses for fixed $n$ as a random variable, so that $B(n) = E[X]$. Then using the above result we can write

$$\begin{equation} B(n) = E[X\ |\ t = g]P(t\ = g) + E[X\ |\ t\ < g]P(t\ < g) + E[X\ |\ t > g]P(t > g)\end{equation}$$

Now, since $t$ is guessed at random, we're using the uniform distribution so 

$$\begin{align}P(t\ = g) &= \frac{1}{n}\\ P(t < g) &= \frac{g - 1}{n}\\ P(t > g) &= \frac{n - g}{n}\end{align}$$

and it's not hard to see that 

$$\begin{align} E[X\ |\ t\ = g] &= 1\\ E[X\ |\ t\ < g] &= B(g - 1) + 1 \\ E[X\ |\ t\ < g] &= B(n - g) + 1\end{align}$$

so we can rewrite $B(n)$

$$\begin{equation} B(n) = \frac{1}{n} + \frac{g - 1}{n}\big(B(g - 1) + 1\big) + \frac{n - g}{n}\big(B(n - g) + 1\big) \end{equation}$$

Note this recursion holds for any choice of $g$.

## Computing $B(n)$

Since we have a recursive formula for $B(n)$ we'll use recursion to compute it. We can save a lot of time using memoization. A [cool way to do this in Python](http://ujihisa.blogspot.com/2010/11/memoized-recursive-fibonacci-in-python.html) in Python uses decorators. With the following all-purpose decorator:

In [4]:
def memoize(f):
    d = {}
    def f_dec(*args):
        if args in d:
            return d[args]
        else:
            d[args] = f(*args)
            return d[args]
    return f_dec

We can memoize a recursive definition of `B(n)`

In [5]:
@memoize
def B(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        g = (n + 1) // 2
        return (1/n) + ((g - 1) / n) * (B(g - 1) + 1) + ((n - g) / n) * (B(n - g) + 1)

Running a few tests for sanity checks

In [6]:
for n in range(1, 7):
    print("B({}) = {}".format(round(n, 8), round(B(n), 8)))

B(1) = 1
B(2) = 1.5
B(3) = 1.66666667
B(4) = 2.0
B(5) = 2.2
B(6) = 2.33333333


## A recursive formula for $R(n)$

Again, let $X$ be the expected number of guesses, so that $R(n) = E[X]$. Let $G$ denote the first random choice of $g$ and $T$ the first random choice of $T$. Then again using total probability,

$$\begin{align} E[X] &= \sum_{g = 1}^n E[X| G = g]P(G = g) \\
&= \frac{1}{n}\left(\sum_{g = 1}^n E[X| G = g]\right)\\
 \end{align}$$
 
and as above

$$\begin{align} E[X | G = g] &=(1)\frac{1}{n} + (R(g -1) + 1)\frac{g - 1}{n} + (R(n - g) + 1)\frac{n - g}{n}
 \end{align}$$

(just like (9) above) hence

$$\begin{align} 
R(n) &= \frac{1}{n}\left(\sum_{g = 1}^n \frac{1}{n} + (R(g - 1) + 1)\frac{g - 1}{n} + (R(n - g) + 1)\frac{n - g}{n}\right)\\
&= \frac{1}{n^2}\left(\sum_{g = 0}^{n - 1} 2g(R(g) + 1) + 1\right)
\end{align}$$

Incidentally, this "full history" recurrence can be converted to the "limited history" recurrence:

$$\begin{align} 
R(n) &= \frac{1}{n^2}\left((n^2 - 1)R(n - 1) + 2n - 1\right)
\end{align}$$

## Computing $R(n)$

We'll code the recurrence (15) iteratively (to avoid Python max recursion depth errors). Also, to get a performance boost we'll use [Cython](http://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html)

In [7]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [8]:
%%cython 
def R(long double n):
    cdef long double R, i
    R, i = 1, 1
    while i < n + 1:
        R = (1 / i ** 2) * ((i**2 - 1)*R + 2*i - 1)
        i += 1
    return R

Sanity checks

In [9]:
for n in range(1, 7):
    print("R({}) = {}".format(round(n, 8), round(R(n), 8)))

R(1) = 1.0
R(2) = 1.5
R(3) = 1.88888889
R(4) = 2.20833333
R(5) = 2.48
R(6) = 2.71666667


The moment of truth!

In [72]:
R(10**10) - B(10**10)

11.92412011606923717